In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import numpy as np
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

df = pd.read_csv('Iowa_Liquor_Sales_old.csv')
df_names = df[['Item Number','Item Description']].drop_duplicates(keep='first', subset=['Item Number'])
# print(df)
# print(df_names)


C:\Users\Maxca\AppData\Local\Temp\ipykernel_6288\3827755466.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Iowa_Liquor_Sales_old.csv')


In [39]:
# pivotDf = pd.pivot_table(df,index = 'Store Number',columns = 'Item Number',values = 'Bottles Sold',aggfunc ='sum').reset_index().fillna(0).drop('Store Number', axis=1)
pivotDf = pd.pivot_table(df,index = 'Store Number',columns = 'Item Number',values = 'Bottles Sold',aggfunc =lambda x: len(x.unique())).reset_index().fillna(0).drop('Store Number', axis=1)
# pivotDf = pd.pivot_table(df,index = 'Store Number',columns = 'Item Number',values = 'Invoice/Item Number',aggfunc ='count').reset_index().fillna(0).drop('Store Number', axis=1)


pivotDf = pivotDf.T.dot(pivotDf)

cosScore = pd.DataFrame(cosine_similarity(pivotDf))
cosScore.index = pivotDf.index
cosScore.columns = np.array(pivotDf.index)
df_pairs = cosScore.stack().reset_index()
df_pairs = df_pairs.rename(columns={'Item Number': 'AlcoholID1', 'level_1': 'AlcoholID2', 0: 'Score'})
df_pairs.reset_index(drop=True, inplace=True, level=0)
df_pairs = df_pairs[df_pairs['Score'] > 0.2].sort_values('Score', ascending=False)
print(df_pairs)

          AlcoholID1  AlcoholID2     Score
2063484         3438        3438  1.000000
29680148      100309      100309  1.000000
22446860       66861       66861  1.000000
47785556      942426      942426  1.000000
35678304      902560      902560  1.000000
...              ...         ...       ...
49359822      964731      904890  0.200012
30355008      900337      917836  0.200004
34627494      902205      900337  0.200004
43841664      917836      900337  0.200004
30353762      900337      902205  0.200004

[46796095 rows x 3 columns]


In [53]:
print(df_names[df_names['Item Description'].str.contains('Finland')])

         Item Number              Item Description
5793           34326     Finlandia Imp Vodka 80prf
33002          35136          Finlandia Grapefruit
40961         935136    Finlandia Vodka Grapefruit
209821         34318            Finlandia Redberry
273245        904717  Finlandia Wild Berries Vodka
296175         34349     Finlandia Raspberry Vodka
710353        903263     Finlandia Cranberry Vodka
963682        904285      Finlandia Vodka 80 Proof
1053423       903254               Finlandia Vodka
1133196       903388        Finlandia Mango Fusion
4395083       934349     Finlandia Vodka Raspberry
8011298       903286          Finlandia Lime Vodka


In [79]:
tmp = df_pairs[df_pairs['AlcoholID1'] == 34326].sort_values('Score', ascending=False)[:8]
# tmp = df_pairs[df_pairs['AlcoholID1'] == 34326].sort_values('Score', ascending=False)[:10]

tmp2 = df_pairs[(df_pairs['AlcoholID1'].isin(tmp['AlcoholID2'].to_list()))]
    # .sort_values('Score', ascending=False)
tmp2 = tmp2[tmp2['Score'] < 1].sort_values('Score', ascending=False)[:12]

tmp3 = df_pairs[(df_pairs['AlcoholID1'].isin(tmp2['AlcoholID2'].to_list()))].sort_values('Score', ascending=False)[:20]
    # .sort_values('Score', ascending=False)
# tmp3 = tmp3[tmp3['Score'] < 1].sort_values('Score', ascending=False)[:20]


# print(tmp)
# print(tmp2)
tmp = tmp.merge(df_names, how='left', left_on='AlcoholID1', right_on='Item Number').rename(columns={'Item Description':'alc_1_title'}).drop(columns='AlcoholID1')
tmp = tmp.merge(df_names, how='left', left_on='AlcoholID2', right_on='Item Number').rename(columns={'Item Description':'alc_2_title'}).drop(columns='Item Number_y').rename(columns={'Item Number_x': 'AlcoholID1'})

tmp2 = tmp2.merge(df_names, how='left', left_on='AlcoholID1', right_on='Item Number').rename(columns={'Item Description':'alc_1_title'}).drop(columns='AlcoholID1')
tmp2 = tmp2.merge(df_names, how='left', left_on='AlcoholID2', right_on='Item Number').rename(columns={'Item Description':'alc_2_title'}).drop(columns='Item Number_y').rename(columns={'Item Number_x': 'AlcoholID1'})

tmp3 = tmp3.merge(df_names, how='left', left_on='AlcoholID1', right_on='Item Number').rename(columns={'Item Description':'alc_1_title'}).drop(columns='AlcoholID1')
tmp3 = tmp3.merge(df_names, how='left', left_on='AlcoholID2', right_on='Item Number').rename(columns={'Item Description':'alc_2_title'}).drop(columns='Item Number_y').rename(columns={'Item Number_x': 'AlcoholID1'})


json_object = tmp.to_json(orient='records')
with open("tmp.json", "w") as outfile:
    outfile.write(json_object)
    
json_object = tmp2.to_json(orient='records')
with open("tmp2.json", "w") as outfile:
    outfile.write(json_object)

json_object = tmp3.to_json(orient='records')
with open("tmp3.json", "w") as outfile:
    outfile.write(json_object)


In [30]:
print(df_pairs[df_pairs['Item Number'] == 35914].sort_values('Score', ascending=False))

          Item Number  AlcoholID2     Score
11641304        35914       35914  1.000000
11640324        35914       11773  0.991159
11641303        35914       35913  0.990682
11642685        35914       65194  0.990537
11640631        35914       23824  0.989128
...               ...         ...       ...
11642474        35914       59153  0.202397
11646100        35914      940658  0.201497
11639816        35914         494  0.201211
11641493        35914       37528  0.201184
11645989        35914      935743  0.200785

[6695 rows x 3 columns]


In [29]:
df_names = pd.read_csv('data/movies.csv')

df_p = df_pairs.copy()

df_p = df_p.merge(df_names, how='inner', left_on='MovieId1', right_on='movieId').rename(columns={'title':'movie_1_title','genres':'movie_1_genres'}).drop(columns='movieId')
df_p = df_p.merge(df_names, how='inner', left_on='MovieId2', right_on='movieId').rename(columns={'title':'movie_2_title','genres':'movie_2_genres'}).drop(columns='movieId')



print(df_p)

          MovieId1  MovieId2     Score                 movie_1_title          movie_1_genres        movie_2_title        movie_2_genres
0             3909      3909  1.000000           Woman on Top (2000)          Comedy|Romance  Woman on Top (2000)        Comedy|Romance
1             2203      3909  0.761385      Shadow of a Doubt (1943)    Crime|Drama|Thriller  Woman on Top (2000)        Comedy|Romance
2             6268      3909  0.806856  Raising Victor Vargas (2002)    Comedy|Drama|Romance  Woman on Top (2000)        Comedy|Romance
3              634      3909  0.355600           Theodore Rex (1995)                  Comedy  Woman on Top (2000)        Comedy|Romance
4             3102      3909  0.648235            Jagged Edge (1985)  Crime|Romance|Thriller  Woman on Top (2000)        Comedy|Romance
...            ...       ...       ...                           ...                     ...                  ...                   ...
80191709      3719      3567  0.447328   Love's 

In [14]:
print(df_names)

      movieId                                      title                                       genres
0           1                           Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1           2                             Jumanji (1995)                   Adventure|Children|Fantasy
2           3                    Grumpier Old Men (1995)                               Comedy|Romance
3           4                   Waiting to Exhale (1995)                         Comedy|Drama|Romance
4           5         Father of the Bride Part II (1995)                                       Comedy
...       ...                                        ...                                          ...
9737   193581  Black Butler: Book of the Atlantic (2017)              Action|Animation|Comedy|Fantasy
9738   193583               No Game No Life: Zero (2017)                     Animation|Comedy|Fantasy
9739   193585                               Flint (2017)                          

In [30]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

print(df_p[df_p['MovieId1'] == 1].sort_values('Score', ascending=False).to_excel('result.xlsx'))

None
